In [1]:
MLFLOW_URI = "https://mlflow.lewagon.ai/"
EXPERIMENT_NAME = "UK Lon lwb_smr vertex_run_02" # template
EXPERIMENT_TAGS = {
    'USER': 'hsth',
    'RUN NAME': 'vertex2, operational',
    'VERSION': 'M2_R04_15',
    'DESCRIPTION': 'Model VGG16 UNet, 20+now another 30 epochs, 72k images',
    'LOSS': 'dice',
    'METRICS': 'accuracy, binaryIoU, AUC'
}

UNET_INPUT_SHAPE = (224,224,3)
BATCH_SIZE = 60
EPOCHS = 30
# LOSS='binary_crossentropy'
LOSS = 'DICE'

# TENSOR SLICE DATA LOADER

In [2]:
import os
import pandas as pd
from lwb_smr.CustomDataLoader import CustomDataLoader
from lwb_smr.data import LoadDataSets
import os
from pathlib import Path
import tensorflow as tf
import matplotlib.pyplot as plt
import random
from PIL import Image, ImageDraw


In [3]:
!ls '../../../raw_data/'


checkpoints		models		      train_mask_tiles_jpeg
image_datasets_csv	models_a	      vertex_data_jpegs.zip
image_datasets_csv.zip	train_RGB_tiles_jpeg


In [4]:
import os
import pandas as pd

x_path = '../../../raw_data/train_RGB_tiles_jpeg/'
x_images = os.listdir(x_path)
y_path = '../../../raw_data/train_mask_tiles_jpeg/'
y_masks = os.listdir(y_path)
        

In [5]:
root_path = '../../../raw_data/'
folders = ['train_RGB_tiles_jpeg', 'train_mask_tiles_jpeg']
folder_path = [f'{root_path}{folder}' for folder in folders]
folder_path

['../../../raw_data/train_RGB_tiles_jpeg',
 '../../../raw_data/train_mask_tiles_jpeg']

In [6]:
train_images, train_mask = [], []
train_images =[f'../../../raw_data/train_RGB_tiles_jpeg/{filename}' for filename in os.listdir(folder_path[0])]
# for i, filename in enumerate(os.listdir(folder_path[0])):
#     if i == 2*BATCH_SIZE: break
#     train_images.append(f'../../../raw_data/train_RGB_tiles_jpeg/{filename}')
    
train_mask = [f'../../../raw_data/train_mask_tiles_jpeg/{filename}' for filename in os.listdir(folder_path[1])]
# for i, filename in enumerate(os.listdir(folder_path[1])):
#     if i == 2*BATCH_SIZE: break
#     train_mask.append(f'../../../raw_data/train_mask_tiles_jpeg/{filename}')
    

In [7]:
train_images.sort()
train_mask.sort()


train_df = pd.DataFrame()
# train_df['file_path'] = train_images
train_df['image_path'] = train_images
train_df['mask_path'] = train_mask

len(train_df), train_df.head()

(72000,
                                           image_path  \
 0  ../../../raw_data/train_RGB_tiles_jpeg/austin1...   
 1  ../../../raw_data/train_RGB_tiles_jpeg/austin1...   
 2  ../../../raw_data/train_RGB_tiles_jpeg/austin1...   
 3  ../../../raw_data/train_RGB_tiles_jpeg/austin1...   
 4  ../../../raw_data/train_RGB_tiles_jpeg/austin1...   
 
                                            mask_path  
 0  ../../../raw_data/train_mask_tiles_jpeg/austin...  
 1  ../../../raw_data/train_mask_tiles_jpeg/austin...  
 2  ../../../raw_data/train_mask_tiles_jpeg/austin...  
 3  ../../../raw_data/train_mask_tiles_jpeg/austin...  
 4  ../../../raw_data/train_mask_tiles_jpeg/austin...  )

In [8]:
# train_df.info()

In [9]:
# root_path = '/raw_data/'

# # raw_data/train_mask_tiles_jpeg

# x_path = root_path + 'train_RGB_tiles_jpeg/'
# x_images = os.listdir(x_path)
# y_path = root_path + 'train_mask_tiles_jpeg/'
# y_masks = os.listdir(y_path)
    

# folders = ['train_RGB_tiles_jpeg', 'train_mask_tiles_jpeg']
# folder_path = [f'{root_path}{folder}' for folder in folders]
# folder_path

# train_images, train_mask = [], []
# train_images = [f'{x_images}{filename}' for filename in os.listdir(folder_path[0])]
# train_mask = [f'{y_masks}{filename}' for filename in os.listdir(folder_path[1])]

# train_images.sort()
# train_mask.sort()


# train_df = pd.DataFrame()
# # train_df['file_path'] = train_images
# train_df['image_path'] = train_images
# train_df['mask_path'] = train_mask

# train_df.head()

In [10]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

def holdout(df, train_ratio=0.8, test_to_val_ratio=0.5, include_all=False):

    img_paths = df["image_path"].values
    msk_paths = df["mask_path"].values

    df_mask = df.copy()

    df_train, df_val = train_test_split(df_mask, train_size=train_ratio)
    df_test, df_val = train_test_split(df_val, test_size=test_to_val_ratio)

    ds_train = tf.data.Dataset.from_tensor_slices(
         (df_train["image_path"].values, df_train["mask_path"].values)
    )
    ds_val = tf.data.Dataset.from_tensor_slices(
        (df_val["image_path"].values, df_val["mask_path"].values)
    )
    ds_test = tf.data.Dataset.from_tensor_slices(
        (df_test["image_path"].values, df_test["mask_path"].values)
    )

    return ds_train, ds_val, ds_test

ds_train, ds_val, ds_test = holdout(train_df)

2022-06-12 14:06:05.505342: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-12 14:06:05.517347: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-12 14:06:05.517969: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-12 14:06:05.519401: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [11]:
def process_path(input_path, mask_path):
    """
    Load images from files.
    :input_path: the path to the satellite file
    :mask_path: the path to the mask file
    :return: The image and mask
    .. note:: Works with jpg images 
              Only the first channel is kept for the mask
    """
    
    IMAGE_SQ_SIZE = 224

    input_img = tf.io.read_file(input_path)   
    input_img = tf.io.decode_jpeg(input_img, channels=3)
    input_img =  tf.image.resize(input_img, [IMAGE_SQ_SIZE, IMAGE_SQ_SIZE])

    mask_img = tf.io.read_file(mask_path)   
    mask_img = tf.io.decode_jpeg(mask_img, channels=1)
    mask_img =  tf.image.resize(mask_img, [IMAGE_SQ_SIZE, IMAGE_SQ_SIZE])

   
    return input_img, mask_img

def normalize(image, mask):
    # image = tf.cast(image, tf.float32) / 255.

    return tf.math.divide(image, 255), tf.math.divide(mask, 255) 

In [12]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

ds_train = ds_train.map(process_path) \
.map(normalize) \
.batch(batch_size=BATCH_SIZE) \
.prefetch(buffer_size=AUTOTUNE)

ds_val = ds_val.map(process_path) \
.map(normalize) \
.batch(batch_size=BATCH_SIZE) \
.prefetch(buffer_size=AUTOTUNE)

### we do not use:
# CustomDataLoader

In [14]:
# data.py

# model.py

#### SMR_Model below, but these cells load the model:


# LOAD THE MODEL SPECIFIED:

In [16]:
# model_load_string # from cells above... RENAMED BELOW ON GCP...

In [17]:
def dice_loss(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.math.sigmoid(y_pred)
    numerator = 2 * tf.reduce_sum(y_true * y_pred)
    denominator = tf.reduce_sum(y_true + y_pred)

    return 1 - numerator / denominator

In [18]:
# # WORKS WELL FOR LOCAL LOAD:
# from tensorflow.keras.models import Model, Sequential

# custom_objects_dict = {
#                 'dice_loss': dice_loss
# }
# loaded_model = tf.keras.models.load_model(model_load_string, custom_objects=custom_objects_dict)

## load direct from GCP bucket:

In [19]:
# NOT THIS ONE!!!!   
#               LOADING_MODEL_GSUTIL = 'gs://lwb-solar-my-roof/models/220610_full_model_vgg16_10e_20e_more.h5'
# THIS ONE >>>>>>> 
#               i.e. with the dice loss
LOADING_MODEL_GSUTIL = 'gs://lwb-solar-my-roof/models/220611_VGG16_Dice_20e_in_shape_224x224x3.h5'

custom_objects_dict = {
                'dice_loss': dice_loss
}
loaded_model = tf.keras.models.load_model(LOADING_MODEL_GSUTIL, custom_objects=custom_objects_dict)


In [20]:
# Check its architecture
loaded_model.summary()

Model: "first_VGG16_UNET"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 224, 224, 64  1792        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv2 (Conv2D)          (None, 224, 224, 64  36928       ['block1_conv1[0][0]']           
                                )                                                  

In [21]:
# from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Activation, ReLU
# from tensorflow.keras.layers import BatchNormalization, Conv2DTranspose, Concatenate
# from tensorflow.keras.models import Model, Sequential
# from tensorflow.keras.utils import plot_model

# from tensorflow.keras.applications import VGG16




# class SMR_Model():
#     ''' creating our first lwb_smr models '''

#     def __init__(self, input_shape):
#         self.input_shape = input_shape

#     def get_latest_model(self):
#         model = self.build_vgg16_unet(self.input_shape)
#         model = self.compile_model(model)

#         return model

#     def convolution_block(self, inputs, num_filters):
#         ''' simple UNET convolution block with BatchNormalisation '''

#         # convolution layer 1 of the block
#         x = Conv2D(num_filters, (3,3), padding='same')(inputs)  # padding='same' to avoid cut-down with conv
#         x = BatchNormalization()(x)
#         x = Activation('relu')(x)

#         # convolution layer 2 of the block
#         x = Conv2D(num_filters, (3,3), padding='same')(x)
#         x = BatchNormalization()(x)
#         x = Activation('relu')(x)

#         # max pooling not used here as just the bridge

#         return x

#     def decoder_block(self, inputs, skip_tensor, num_filters):
#         ''' decoder block for UNET '''
#         # adds in the skips with concatenate
#         x = Conv2DTranspose(num_filters, (2,2), strides=2, padding='same')(inputs) # stride important here to up-sample
#         x = Concatenate()([x, skip_tensor])     # bringing in skip layer
#         x = self.convolution_block(x, num_filters)

#         return x

#     def build_vgg16_unet(self, input_shape):
#         ''' build vgg-16 '''

#         inputs = Input(input_shape)

#         # see actual VGG-16 here: https://github.com/keras-team/keras/blob/v2.9.0/keras/applications/vgg16.py#L43-L227
#         vgg16 = VGG16(include_top=False, weights='imagenet', input_tensor=inputs)
#         # vgg16.summary()
#         vgg16.trainable = False

#         ''' Encoder - skip layers '''
#         skip1 = vgg16.get_layer('block1_conv2').output #  256 x 256, 64 filters in vgg16
#         skip2 = vgg16.get_layer('block2_conv2').output #  128 x 128, 128 filters in vgg16
#         skip3 = vgg16.get_layer('block3_conv3').output #   64 x 64, 256 filters in vgg16
#         skip4 = vgg16.get_layer('block4_conv3').output #   32 x 32, 512 filters in vgg16
#         # display('skip4: ' + str(skip4.shape))

#         # only need to specify the skip layers, as VGG16 is an Encoder
#         # Therefore, VGG16 comes built with MaxPool2d, so we don't specify

#         ''' Bridge '''
#         bridge = vgg16.get_layer('block5_conv3').output # 16 x 16, with 512 filters in vgg16
#         # display('bridge: ' + str(bridge.shape))


#         ''' Decoder '''
#         d1 = self.decoder_block(bridge, skip4, 512) #  512 filters, as per the bridge
#         d2 = self.decoder_block(d1, skip3, 256) #  256 filters
#         d3 = self.decoder_block(d2, skip2, 128) #  128 filters
#         d4 = self.decoder_block(d3, skip1, 64)  #   64 filters

#         ''' Output '''
#         outputs = Conv2D(1, (1,1), padding='same', activation='sigmoid')(d4)

#         model = Model(inputs, outputs, name='first_VGG16_UNET')

#         return model
    
#     def dice_loss(self, y_true, y_pred):
#         y_true = tf.cast(y_true, tf.float32)
#         y_pred = tf.math.sigmoid(y_pred)
#         numerator = 2 * tf.reduce_sum(y_true * y_pred)
#         denominator = tf.reduce_sum(y_true + y_pred)

#         return 1 - numerator / denominator


#     def compile_model(self, m):
#         ''' with accuracy, binaryIoU, AuC '''
#         # metrics
#         threshold = 0.5
#         binaryIoU = tf.keras.metrics.BinaryIoU(target_class_ids=[1], threshold=threshold)
#         AuC = tf.keras.metrics.AUC()

#         # loss
#         #self.dice_loss = ...
        
#         # Compile Model
#         m.compile(
#                     loss=self.dice_loss,
#                     optimizer='adam',
#                     metrics=['accuracy', binaryIoU, AuC]
#                     )
#         return m


# utils.py

In [22]:
# mlflow
#
# and others...

import mlflow
from mlflow.tracking import MlflowClient
from memoized_property import memoized_property

MLFLOW_URI = "https://mlflow.lewagon.ai/"

class PushMLFlow():
    '''
        MLFLOW_URI = "https://mlflow.lewagon.ai/"
        EXPERIMENT_NAME = "[UK] [LONDON] [SOLAR_ROOF] TEST RUN" # template
        EXPERIMENT_TAGS = {
            'USER': 'test_user',
            'RUN NAME': 'test runs',
            'VERSION': '1.0.1',
            'LOSS': 'dice'
            'DESCRIPTION': 'testing MLFlow Pipeline. Model - basic U-Net structure, 2 epochs, 15 images'
        }
    '''

    def __init__(self, experiment_name, experiment_tags):
        self.experiment_name = experiment_name
        self.experiment_tag = experiment_tags

    @memoized_property
    def mlflow_client(self):
        mlflow.set_tracking_uri(MLFLOW_URI)
        return MlflowClient()

    @memoized_property
    def mlflow_experiment_id(self):
        try:
            return self.mlflow_client.create_experiment(self.experiment_name)
        except BaseException:
            return self.mlflow_client.get_experiment_by_name(self.experiment_name).experiment_id

    @memoized_property
    def mlflow_run(self):
        return self.mlflow_client.create_run(self.mlflow_experiment_id, tags=self.experiment_tag)

    def mlflow_log_param(self, key, value):
        self.mlflow_client.log_param(self.mlflow_run.info.run_id, key, value)

    def mlflow_log_metric(self, key, value):
        self.mlflow_client.log_metric(self.mlflow_run.info.run_id, key, value)


In [23]:
# trainer.py

In [ ]:

import os

import tensorflow as tf

# from lwb_smr.CustomDataLoader import CustomDataLoader
# from lwb_smr.model import SMR_Model
# from lwb_smr.utils import PushMLFlow

from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.metrics import AUC, IoU

        
def dice_loss(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.math.sigmoid(y_pred)
    numerator = 2 * tf.reduce_sum(y_true * y_pred)
    denominator = tf.reduce_sum(y_true + y_pred)

    return 1 - numerator / denominator

class Trainer():
    def __init__(self):
        pass

    def set_model(self, loss=dice_loss):
        self.loss = loss
        # Instantiate Model
        # our_input_shape = (224,224,3)

        #### getVGG16 = SMR_Model(UNET_INPUT_SHAPE)
        #### self.model = getVGG16.get_latest_model()
        self.model = loaded_model # see additions above
        # see compile in SMR_Model

    def start_mlflow(self):
        p = PushMLFlow(EXPERIMENT_NAME, EXPERIMENT_TAGS)
        return p # returns a class instance of PushMLFlow
    
    def run(self):

        print(80*'-')
        print('------SETTING FOR DATA RUN------')

        ### 
        ##
        #
        # customdata = self.just_get_the_data_loaded()

        print(80*'-')
        print('------MODEL RUNNING------')

        # set mflow      
        self.MFLOW = self.start_mlflow() # class instance of MLFLOW

        
        # set model
        self.set_model()

        mc = ModelCheckpoint('220612_checkpoint_VGG16_Dice.h5', save_best_only=True) # could put path here
        es = EarlyStopping(patience=15, restore_best_weights=True)
        self.history = self.model.fit(
            ds_train,
            validation_data=ds_val,
            batch_size=BATCH_SIZE,
            epochs=EPOCHS,
            callbacks=[mc, es]
            )

        model_path_and_filename = '220612_VGG16_Dice_20e_now30e_more_in_shape_224x224x3.h5'
        self.model.save(model_path_and_filename)

        self.MFLOW.mlflow_log_param('loss', self.loss)

        print(80*'=')
        print('------MODEL RUN SUCCESFULLY COMPLETED------')

        self.evaluate()

    def evaluate(self):
        print(80*'-')
        print('------MODEL EVALUATING------')
        results = self.model.evaluate(ds_val)
        for result in results:
            self.MFLOW.mlflow_log_metric('metric X val output', result)
        print(80*'=')
        print('------MODEL EVALUATED------')

if __name__ == '__main__':
    pass
    t = Trainer()
    t.run()

--------------------------------------------------------------------------------
------SETTING FOR DATA RUN------
--------------------------------------------------------------------------------
------MODEL RUNNING------
Epoch 1/30


2022-06-12 14:06:35.175788: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8200
2022-06-12 14:06:56.926811: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.92GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2022-06-12 14:06:56.926880: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.92GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.


In [ ]:
########
####                        DICE             DICE             DICE             DICE             DICE             DICE             DICE             DICE
###
#
#        NOTES: with DICE LOSS driving the model now (this is a sample run. 
#                                                                          Again, clear that binaryIoU metric is showing increases, as with all other metrics.
#                                              (val) DICE LOSS reducing
#                                              (val) ACCURACY increasing
#                                              (val) binaryIoU increasing
#                                              (val) AUC increasing
#
##
###
####
########

In [ ]:
t.history

In [ ]:
t.__dict__